In [1]:
import pandas as pd
import unidecode


<h2>Juntar datos de alcades en un único dataframe</h2>

In [45]:
def cargaAlcaldes(archivo, columna, fila, cabecera, nombreHoja):
    return pd.read_excel(archivo, sheet_name=nombreHoja,
         names=cabecera, usecols=columna, skiprows=fila)

directorio = fr"C:\Users\alejandro.regueiroru\Desktop\Alcaldes/"

archivos = [
    [directorio + 'Alcaldes_Mandato_1979_1983.xlsx', 'D:K', 7, 
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1979-1983'],
    [directorio + 'Alcaldes_Mandato_1983_1987.xlsx', 'D:K', 7, 
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1983-1987'],
    [directorio + 'Alcaldes_Mandato_1987_1991.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1987-1991'],
    [directorio + 'Alcaldes_Mandato_1991_1995.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1991-1995'],
    [directorio + 'Alcaldes_Mandato_1995_1999.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1995-1999'],
    [directorio + 'Alcaldes_Mandato_1999_2003.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 1999-2003'],
    [directorio + 'Alcaldes_Mandato_2003_2007.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 2003-2007'],
    [directorio + 'Alcaldes_Mandato_2007_2011.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 2007-2011'],
    [directorio + 'Alcaldes_Mandato_2011_2015.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 2011-2015'],
    [directorio + 'Alcaldes_Mandato_2015_2019.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 2015-2019'],
    [directorio + 'Alcaldes_Mandato_2019_2023.xlsx', 'D:K', 7,
        ["COMUNIDAD AUTÓNOMA", "PROVINCIA", "MUNICIPIO", "CÓDIGO INE", "NOMBRE", "LISTA", "FECHA POSESIÓN", "FECHA BAJA"], 'Alcaldes 2015-2019']
]

#Inicialización do dataframe vacío
dfAlcaldes = pd.DataFrame([])

for i in range(0, len(archivos)):
    dfAlcaldes = pd.concat([dfAlcaldes,
        cargaAlcaldes(archivos[i][0], archivos[i][1], 
            archivos[i][2], archivos[i][3], archivos[i][4]),]
    )

# Fecha de posesion formato año-mes-dia
dfAlcaldes['FECHA POSESIÓN'] = pd.to_datetime(dfAlcaldes['FECHA POSESIÓN'], format='%d/%m/%Y', errors='coerce')

# Fecha de baja formato año-mes-dia
dfAlcaldes['FECHA BAJA'] = pd.to_datetime(dfAlcaldes['FECHA BAJA'], format='%d/%m/%Y', errors='coerce')


<h2>Cuantas veces repitió mandato cada alcalde en base a su nombre registrado</h2>

In [ ]:
# Saber qué alcaldes repetiron mandato e en qué lexislaturas
mandatos = dfAlcaldes.groupby('NOMBRE').size().reset_index(name='Veces en el cargo').sort_values(by='Veces en el cargo', ascending=False)
mandatos = mandatos[mandatos['Veces en el cargo'] > 1]

# Restar 1 para saber cuántas legislaturas repitió
mandatos['Veces en el cargo'] = mandatos['Veces en el cargo'] - 1

# Añadir las fechas de posesión para cada alcalde formato año-mes-dia
fechas_mandato = pd.DataFrame()
fechas_mandato = dfAlcaldes.groupby('NOMBRE')['FECHA POSESIÓN'].apply(list).reset_index(name='FECHA POSESIÓN')
mandatos = mandatos.merge(fechas_mandato, on='NOMBRE')

# Nos quedamos solo con los años de posesión
mandatos['FECHA POSESIÓN'] = mandatos['FECHA POSESIÓN'].apply(lambda x: [i.year for i in x])

mandatos


,NOMBRE,Veces en el cargo,FECHA POSESIÓN
0,"PEREZ PEREZ, JESUS",12,"[1979, 1979, 1983, 1983, 1987, 1987, 1987, 199..."
1,"MARTINEZ MARTINEZ, ANTONIO",11,"[1979, 1983, 1985, 1983, 1983, 1983, 1987, 198..."
2,"MARTIN SANCHEZ, JOSE",11,"[1979, 1983, 1987, 1987, 1991, 1991, 1995, 199..."
3,"GARCIA GARCIA, MANUEL",10,"[1983, 1983, 1987, 1987, 1991, 1991, 1991, 199..."
4,"MARTIN PEREZ, JOSE",8,"[1983, 1987, 1987, 1991, 1991, 1995, 1995, 199..."
...,...,...,...
19436,"VECINO FERNANDEZ, TEODORO",1,"[1987, 1991]"
19437,"DIAZ SANCHEZ-CABEZUDO, MARCELIANO",1,"[1995, 1999]"
19438,"DIAZ VILCHES, LUIS GREGORIO",1,"[1979, 1999]"
19439,"CARRERAS PUIGDELLIURA, ANDREU",1,"[1987, 1999]"


<h2>Partidos con más éxito(Contamos el número de alcaldes de cada partido)</h2>

In [52]:
# Determinar qué partidos políticos tuvieron más éxito, entre las personas que repiteiron mandato
partidos_exito  = dfAlcaldes[dfAlcaldes['NOMBRE'].isin(mandatos['NOMBRE'])]['LISTA'].value_counts().reset_index()
partidos_exito.columns = ['PARTIDO', 'NUMERO DE ALCALDES']
partidos_exito = partidos_exito.sort_values(by='NUMERO DE ALCALDES', ascending=False)
partidos_exito

,PARTIDO,NUMERO DE ALCALDES
0,PSOE,16796
1,PP,15766
2,IND,3180
3,CIU,2327
4,AP,1479
...,...,...
87,CS,1
88,PPSO,1
89,CC/ATI,1
90,C. GEST,1


<h2>Muestra todos los alcaldes que pertenecieran como mínimo en 2 partidos distintos</h2>

In [43]:
# Determinar qué alcaldes cambiaron de partido
alcaldes_partido = dfAlcaldes.groupby('NOMBRE')['LISTA'].nunique().reset_index()
alcaldes_partido.columns = ['NOMBRE', 'NUMERO_DE_PARTIDOS']
alcaldes_partido = alcaldes_partido[alcaldes_partido['NUMERO_DE_PARTIDOS'] >= 2].sort_values(by='NUMERO_DE_PARTIDOS', ascending=False)
alcaldes_partido

,NOMBRE,NUMERO_DE_PARTIDOS
51547,"PEREZ PEREZ, JESUS",7
42720,"MARTINEZ RODRIGUEZ, JOSE",6
17011,"FERNANDEZ FERNANDEZ, PEDRO",6
4886,"ARIAS MAYOR, GREGORIO",5
42358,"MARTINEZ CRESPO, AMBROSIO",5
...,...,...
23230,"GONZALEZ TIERNO, BENITO",2
23220,"GONZALEZ SOTES, FERNANDO",2
23209,"GONZALEZ SANTOS, EUSEBIO",2
23199,"GONZALEZ SANCHEZ, TOMAS",2


<h2>Alcaldes que cesaron legislatura antes de tiempo</h2>

In [46]:
cese = dfAlcaldes[dfAlcaldes['FECHA BAJA'].notnull()]
cese_repetidas = cese.groupby('NOMBRE').size().reset_index(name='Veces cesadas').sort_values(by='Veces cesadas', ascending=False)
cese

,COMUNIDAD AUTÓNOMA,PROVINCIA,MUNICIPIO,CÓDIGO INE,NOMBRE,LISTA,FECHA POSESIÓN,FECHA BAJA
37,Andalucía,Almería,DALIAS,040388,"GARCIA ACIEN, JOSE ANTONIO",UCD,1982-07-30,1982-11-20
38,Andalucía,Almería,DALIAS,040388,"MARTIN MALDONADO, LUIS",PSOE,1979-04-19,1982-07-30
106,Andalucía,Cádiz,ALCALA DE LOS GAZULES,110012,"AIDO ARROYO, FRANCISCO",PSOE,1979-04-19,1981-04-17
107,Andalucía,Cádiz,ALCALA DE LOS GAZULES,110012,"ROMERO ACEDO, LUIS",PSOE,1981-04-17,1983-03-30
112,Andalucía,Cádiz,ALGODONALES,110051,"VALLE ESCORZA, DIOGENES",PSOE,1979-04-19,1981-03-17
...,...,...,...,...,...,...,...,...
8705,"Rioja, La","Rioja, La",Robres del Castillo,261269,Ricardo García Barrio,PP,2019-06-15,2022-07-30
8714,"Rioja, La","Rioja, La",San Vicente de la Sonsierra,261428,María Isabel Bañares Ortún,PP,2019-06-15,2021-01-25
8719,"Rioja, La","Rioja, La",Santo Domingo de la Calzada,261389,Javier Ruiz Bartolomé,PSOE,2019-06-15,2019-10-16
8729,"Rioja, La","Rioja, La",Tobía,261490,Luis Alberto Rubio Llerena,PSOE,2019-06-15,2020-04-11


<h2> Sueldos de alcaldes años(2019-2023)</h2>

In [ ]:
sueldos_df = pd.DataFrame()
sueldos_df = pd.read_excel(fr"C:\Users\alejandro.regueiroru\Desktop\Alcaldes\Alcaldes_Mandato_2019_2023.xlsx", usecols="D:K", skiprows=7)
sueldos_df.columns = ['COMUNIDAD AUTÓNOMA', 'PROVINCIA', 'MUNICIPIO', 'CÓDIGO INE', 'NOMBRE', 'LISTA', 'FECHA POSESIÓN', 'FECHA BAJA']

sueldos_df = sueldos_df.dropna(subset=['COMUNIDAD AUTÓNOMA'])
sueldos_df = sueldos_df.dropna(subset=['NOMBRE'])
sueldos_df = sueldos_df[sueldos_df['COMUNIDAD AUTÓNOMA'] != 'COMUNIDAD AUTÓNOMA']

sueldos = pd.read_csv("tabula-retribuciones_alcaldes.pdf.csv")

# Normalizar los datos de las columnas MUNICIPIO, PROVINCIA, COMUNIDAD AUTÓNOMA

sueldos_df['MUNICIPIO'] = sueldos_df['MUNICIPIO'].str.strip().str.upper().apply(unidecode.unidecode)
sueldos_df['PROVINCIA'] = sueldos_df['PROVINCIA'].str.strip().str.upper().apply(unidecode.unidecode)
sueldos_df['COMUNIDAD AUTÓNOMA'] = sueldos_df['COMUNIDAD AUTÓNOMA'].str.strip().str.upper().apply(unidecode.unidecode)

sueldos['MUNICIPIO'] = sueldos['MUNICIPIO'].str.strip().str.upper().apply(unidecode.unidecode)
sueldos['PROVINCIA'] = sueldos['PROVINCIA'].str.strip().str.upper().apply(unidecode.unidecode)
sueldos['COMUNIDAD AUTÓNOMA'] = sueldos['COMUNIDAD AUTÓNOMA'].str.strip().str.upper().apply(unidecode.unidecode)

# Adjudicar la retribución correspondiente a cada fila donde coincidan los 3 datos
sueldos_df = sueldos_df.merge(sueldos, on=['MUNICIPIO', 'PROVINCIA', 'COMUNIDAD AUTÓNOMA'], how='left')

# Convertir la columna RETRIBUCIÓN a numérica
sueldos_df['RETRIBUCIÓN'] = sueldos_df['RETRIBUCIÓN'].str.replace('.', '').str.replace(',', '.').astype(float)

sueldos_df = sueldos_df.sort_values(by='RETRIBUCIÓN', ascending=False)

sueldos_df

,COMUNIDAD AUTÓNOMA,PROVINCIA,MUNICIPIO,CÓDIGO INE,NOMBRE,LISTA,FECHA POSESIÓN,FECHA BAJA,DEDICACIÓN,RETRIBUCIÓN
7833,"MADRID, COMUNIDAD DE",MADRID,MADRID,280796,José Luis Martínez-Almeida Navasqües,PP,15/06/2019,NaN,Exclusiva,108517.80
5367,CATALUNA,BARCELONA,BARCELONA,80193,ADA COLAU BALLANO,C. ELECTORAL,15/06/2019,NaN,Exclusiva,100000.04
8385,PAIS VASCO,BIZKAIA,BILBAO,480209,JUAN MARIA ABURTO RIKE,EAJ-PNV,15/06/2019,NaN,Exclusiva,98018.40
8352,PAIS VASCO,ARABA/ALAVA,VITORIA-GASTEIZ,10590,GORKA URTARAN AGIRRE,EAJ-PNV,15/06/2019,NaN,Exclusiva,91323.33
853,ANDALUCIA,SEVILLA,SEVILLA,410917,JUAN ESPADAS CEJAS,PSOE,15/06/2019,2021-12-20 00:00:00,Exclusiva,88673.68
...,...,...,...,...,...,...,...,...,...,...
8677,"RIOJA, LA","RIOJA, LA",MURILLO DE RIO LEZA,260990,SERGIO CHICOTE BELTRÁN,PSOE,2021-02-18 00:00:00,NaN,NaN,NaN
8698,"RIOJA, LA","RIOJA, LA",PREJANO,261196,Eduardo Ruiz Cubillo,PSOE,15/06/2019,NaN,NaN,NaN
8705,"RIOJA, LA","RIOJA, LA",ROBRES DEL CASTILLO,261269,Ricardo García Barrio,PP,15/06/2019,2022-07-30 00:00:00,NaN,NaN
8706,"RIOJA, LA","RIOJA, LA",ROBRES DEL CASTILLO,261269,José Ignacio Rincón Del Castillo,PP,2022-11-22 00:00:00,NaN,NaN,NaN


<h2>Relacionar sueldos con probabilidad de repetir mandato</h2>

In [ ]:
# Muestra los alcaldes que repetiron mandato y cuanto cobraron
mandatos = dfAlcaldes.groupby('NOMBRE').size().reset_index(name='Veces en el cargo').sort_values(by='Veces en el cargo', ascending=False)
mandatos = mandatos[mandatos['Veces en el cargo'] > 1]

# Ordenado por número de veces en el cargo
mandatos = mandatos.merge(sueldos_df, on='NOMBRE')
mandatos
# Ordenado por retribución(Parece que no hay relación entre el número de veces en el cargo y la retribución)
mandatos = mandatos.sort_values(by='RETRIBUCIÓN', ascending=False)
mandatos

<h2>Otros datos de interés<h2>

In [ ]:
# Hay relación entre el tipo de dedicación y la retribución? (Vamos a agrupar por tipo de dedicación y hacer una media de la retribución)
interesSueldos_df = sueldos_df.groupby('DEDICACIÓN').agg({'RETRIBUCIÓN': 'mean'}).sort_values(by='RETRIBUCIÓN', ascending=False).reset_index()

# Parece que según el tipo de dedicación, la retribución es mayor o menor
interesSueldos_df

#----------------------------------------------#

# En base al año de la legislatura cual fue el partido que tuvo más alcaldes en el cargo
interesPartidos_df = dfAlcaldes.groupby(['LISTA', dfAlcaldes['FECHA POSESIÓN'].dt.year]).size().reset_index(name='NUMERO DE ALCALDES')
interesPartidos_df = interesPartidos_df.sort_values(by='NUMERO DE ALCALDES', ascending=False)

# Nos quedamos solo con las legislaturas de cada 4 años, a partir de 1979
interesPartidos_df = interesPartidos_df[interesPartidos_df['FECHA POSESIÓN'] % 4 == 3]

# Mostramos el partido que más alcaldes tuvo en cada año
interesPartidos_df = interesPartidos_df.drop_duplicates(subset='FECHA POSESIÓN', keep='first')
interesPartidos_df






,LISTA,FECHA POSESIÓN,NUMERO DE ALCALDES
697,UCD,1979.0,3982
596,PP,2011.0,3810
584,PP,1999.0,3466
580,PP,1995.0,3413
588,PP,2003.0,3275
654,PSOE,1991.0,3214
592,PP,2007.0,3157
600,PP,2015.0,3092
651,PSOE,1987.0,2872
604,PP,2019.0,2865
